<a href="https://colab.research.google.com/github/vinaykanth77/Qa-bot/blob/main/Interactive_QA_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install streamlit cohere pinecone-client pdfplumber sentence-transformers
!pip install pinecone-client sentence-transformers cohere
!pip install pinecone-client cohere
!pip install pdfplumber
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/

In [4]:
import gradio as gr
import pinecone
from sentence_transformers import SentenceTransformer
import cohere
import pdfplumber
import os

# Initialize clients and models and add your own Pinecone and Cohere api key
pinecone_client = pinecone.Pinecone(api_key="7ba597c6-7922-412f--9f5993ed1882", environment="us-west1-gcp-free")
co = cohere.Client('1esBQiIg4ILGYf5FEslCwob4b44Ksdxv')
model = SentenceTransformer('all-MiniLM-L6-v2')

# Connect to Pinecone index
index_name = 'document-qa-index'
index = pinecone_client.Index(index_name)

def process_pdf(pdf_file):
    with pdfplumber.open(pdf_file.name) as pdf:
        document_text = ""
        for page in pdf.pages:
            document_text += page.extract_text()
    return document_text

def insert_document(document_text):
    embeddings = model.encode(document_text)
    index.upsert([(f'doc_{index.describe_index_stats().total_vector_count}', embeddings.tolist(), {'text': document_text})])

def retrieve_and_answer(question):
    query_embed = model.encode([question])
    result = index.query(vector=query_embed.tolist(), top_k=3, include_metadata=True)
    retrieved_docs = [doc['metadata']['text'] for doc in result['matches']]
    prompt = f"Answer the question '{question}' based on the following context: {retrieved_docs}"
    response = co.generate(
        model='command-r-plus',
        prompt=prompt,
        max_tokens=200,
        temperature=0.5,
        k=0
    )
    return response.generations[0].text

def upload_pdf(pdf_file):
    if pdf_file:
        document_text = process_pdf(pdf_file)
        insert_document(document_text)
        return "Document processed and added to the index. You can now ask questions."
    return "No file uploaded. Please upload a PDF file."

def ask_question(question):
    if question:
        answer = retrieve_and_answer(question)
        return answer
    return "Please enter a question."

# Create Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# QA Bot")
    gr.Markdown("Upload a PDF document and ask questions about its content.")

    with gr.Tab("Upload Document"):
        pdf_input = gr.File(label="Upload PDF")
        upload_button = gr.Button("Process Document")
        upload_output = gr.Textbox(label="Upload Status")

        upload_button.click(upload_pdf, inputs=pdf_input, outputs=upload_output)

    with gr.Tab("Ask Questions"):
        question_input = gr.Textbox(label="Ask a question")
        answer_button = gr.Button("Get Answer")
        answer_output = gr.Textbox(label="Answer")

        answer_button.click(ask_question, inputs=question_input, outputs=answer_output)

# Launch the interface
iface.launch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bfc4499d8dcd26a9ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
